In [1]:
import requests
from datetime import datetime, timedelta
from typing import List
import json

from optimization import improve_graph

def get_task_goals(
    base_url: str = "https://stackvm.tidb.ai/",
   last_hours: int = 2
) -> List[str]:
    try:
        end_time = datetime.utcnow()
        start_time = end_time - timedelta(hours=last_hours)
        
        url = f"{base_url}/api/tasks/evaluation"
        params = {
            "start_time": start_time.isoformat(),
            "end_time": end_time.isoformat()
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        tasks = response.json()
        
        goals = [task.get("goal", "") for task in tasks if task.get("goal")]
        
        return goals
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching task goals: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return []
    except Exception as e:
        print(f"Unknown error: {e}")
        return []
import time

while True:
    goals = get_task_goals()
    for goal in goals:
        improve_graph(goal)
    
    # Sleep for 10 minutes before next iteration
    print(f"Sleeping for 10 minutes... Current time: {datetime.now()}")
    time.sleep(600)  # 600 seconds = 10 minutes

/var/folders/n5/0j85cw4x3vggqtqjqlk8k2v00000gn/T/ipykernel_32015/2387476037.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow()


ModuleNotFoundError: No module named 'numpy._core.numeric'